In [1]:
import pandas as pd
import pymorphy2
import nltk
import re

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import datetime
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm
from gensim.models import *
from gensim import corpora
from threading import Thread
from time import sleep

In [2]:
dir_hackathon = '.'
dir_data_in = f'{dir_hackathon}/Data/In'
dir_data_out = f'{dir_hackathon}/Data/Out'

all_data_in = pd.read_excel(f'{dir_data_in}/dateset_testable.xlsx', header=0)
all_data_in.columns = [	'Номер', 'Номер продукции',	'Коды ТН ВЭД ЕАЭС',	'Технические регламенты',
	'Группа продукции',	'Общее наименование продукции',	'ИЛ',	'Заявитель',	'Адрес Заявителя',	'Изготовитель',	'Страна',
    	'Адрес изготовителя',	'is_valid',	'Группа продукции_ordinal',	'Технические регламенты_ordinal',	'Коды ТН ВЭД ЕАЭС_ordinal'
]
all_data_in.head(2)

,Номер,Номер продукции,Коды ТН ВЭД ЕАЭС,Технические регламенты,Группа продукции,Общее наименование продукции,ИЛ,Заявитель,Адрес Заявителя,Изготовитель,Страна,Адрес изготовителя,is_valid,Группа продукции_ordinal,Технические регламенты_ordinal,Коды ТН ВЭД ЕАЭС_ordinal
0,0,37632 Продукция,4016999708,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Узлы пересечения противопожарных преград кабел...,Проходки кабельные модульные универсальные огн...,ИЛ Мопалия Миддендорфа,ООО Европейский гологлаз 6,РОССИЯ 461220 БАРАБАНОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Ара Леара 11,РОССИЯ,РОССИЯ 182375 ЛУКИНО ПСКОВСКАЯ ОБЛАСТЬ,1,1763,3,205
1,1,37565 Продукция,7308909809,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Противопожарные окна, двери, двери шахт лифтов...",Ворота противопожарные металлические распашные...,ИЛ Мопалия Миддендорфа,ООО Дроздовидная6,РОССИЯ 461182 ШЕСТАКОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Аннамский палочник 11,РОССИЯ,РОССИЯ 182306 ПОДДУБЬЕ ПСКОВСКАЯ ОБЛАСТЬ,1,1277,3,3822


In [3]:
column_num = 'Номер'
column_name = 'Общее наименование продукции'
column_group = 'Группа продукции'
data_lemm_out = pd.read_excel(f'{dir_data_out}/data_lemm_morphy.xlsx', header=0)
data_lemm_out.columns = [column_num, column_name, column_group]

data_lemm_out.tail(5)

,Номер,Общее наименование продукции,Группа продукции
66747,66748,электрический электрооборудование неэлектричес...,блок пробоотборный обогревать
66748,66749,электрический электрооборудование неэлектричес...,система отбор проба природный газ
66749,66750,электрический электрооборудование неэлектричес...,фильтр
66750,66751,элемент оборудование сборочный единица комплек...,элемент трубопровод
66751,66752,электрический электрооборудование неэлектричес...,станция газозаправочный автомобильный


In [4]:
data_lemm_out_small = data_lemm_out[:500]
data_lemm_out_small.tail(2)

,Номер,Общее наименование продукции,Группа продукции
498,499,игрушка электрический товар развлечение настол...,игрушка ребёнок старший год пластмасса
499,500,игрушка электрический товар развлечение настол...,игрушка ребёнок старший х игра настольный разв...


In [5]:
column_id_group = 'Группа продукции_ordinal'

#data_new = pd.read_excel(f'{dir_data_in}/dateset_categorical.xlsx', header=0)
data_claster_out_name = data_lemm_out[[column_num, column_name]]
# data_claster_out_gr = data_new[[column_num, column_id_group]]
data_claster_out_gr = all_data_in[[column_num, column_id_group]]
data_claster_out = pd.merge(data_claster_out_name, data_claster_out_gr, on="Номер")
data_claster_out.tail(3)

,Номер,Общее наименование продукции,Группа продукции_ordinal
73424,66750,электрический электрооборудование неэлектричес...,1940
73425,66751,элемент оборудование сборочный единица комплек...,2024
73426,66752,электрический электрооборудование неэлектричес...,1940


In [6]:
data_claster_out[column_name] = data_claster_out[column_name].astype('str')
words = " ".join(data_claster_out[column_name]).split()
print(words[:100])

['противопожарный', 'окно', 'дверь', 'дверь', 'шахта', 'лифт', 'нормировать', 'предел', 'огнестойкость', 'ворота', 'люк', 'штора', 'роллета', 'экран', 'занавес', 'клапан', 'противопожарный', 'нормально', 'открытый', 'клапан', 'противопожарный', 'нормально', 'закрытый', 'люк', 'дымовой', 'вытяжной', 'вентилятор', 'вытяжной', 'вентилятор', 'свитер', 'джемпер', 'жакет', 'юбка', 'брюки', 'костюм', 'рейтузы', 'полукомбинезон', 'комбинезон', 'шорты', 'платье', 'комплект', 'блузка', 'сорочка', 'верхний', 'жилет', 'аналогичный', 'изделие', 'заявить', 'изготовитель', 'предназначить', 'ребёнок', 'подросток', 'пижама', 'кальсоны', 'панталоны', 'фуфайка', 'комбинация', 'купальный', 'изделие', 'трус', 'майка', 'аналогичный', 'изделие', 'заявить', 'изготовитель', 'предназначить', 'ребёнок', 'старший', 'и', 'подросток', 'клапан', 'противопожарный', 'нормально', 'открытый', 'клапан', 'противопожарный', 'нормально', 'закрытый', 'люк', 'дымовой', 'клапан', 'противопожарный', 'нормально', 'открытый', 'кл

In [7]:
data_claster_out[column_name] = data_claster_out[column_name].astype('str')
words = " ".join(data_claster_out[column_name]).split()
print(words[:100])
common_projects_words = Counter(words).most_common(1000)

['противопожарный', 'окно', 'дверь', 'дверь', 'шахта', 'лифт', 'нормировать', 'предел', 'огнестойкость', 'ворота', 'люк', 'штора', 'роллета', 'экран', 'занавес', 'клапан', 'противопожарный', 'нормально', 'открытый', 'клапан', 'противопожарный', 'нормально', 'закрытый', 'люк', 'дымовой', 'вытяжной', 'вентилятор', 'вытяжной', 'вентилятор', 'свитер', 'джемпер', 'жакет', 'юбка', 'брюки', 'костюм', 'рейтузы', 'полукомбинезон', 'комбинезон', 'шорты', 'платье', 'комплект', 'блузка', 'сорочка', 'верхний', 'жилет', 'аналогичный', 'изделие', 'заявить', 'изготовитель', 'предназначить', 'ребёнок', 'подросток', 'пижама', 'кальсоны', 'панталоны', 'фуфайка', 'комбинация', 'купальный', 'изделие', 'трус', 'майка', 'аналогичный', 'изделие', 'заявить', 'изготовитель', 'предназначить', 'ребёнок', 'старший', 'и', 'подросток', 'клапан', 'противопожарный', 'нормально', 'открытый', 'клапан', 'противопожарный', 'нормально', 'закрытый', 'люк', 'дымовой', 'клапан', 'противопожарный', 'нормально', 'открытый', 'кл

In [8]:
data_common_words = pd.DataFrame(list(common_projects_words))
data_common_words.columns = ['Слово', 'Количество ']
data_common_words.to_excel(f'{dir_data_out}/data_common_words.xlsx')
data_common_words.tail(5)

,Слово,Количество
995,слива,60
996,конденсат,60
997,управлять,60
998,ускорительный,60
999,корректировка,60


In [9]:
words = ' '.join(data_common_words['Слово']).split()
# words


In [10]:
# Get top  from clasters
def top_terms(df, top_n=50):
    words = ' '.join(df[column_name]).split()
    count_words = len(words)  
    list_top_words_and_count = Counter(words).most_common(top_n)
    # print(list_top_words_and_count)

    # list_top_words_count = [a_tuple[1] for a_tuple in list_top_words_and_count]
    # list_top_words = [a_tuple[0] for a_tuple in list_top_words_and_count]

    # data_claster_common_words = pd.DataFrame(list(common_projects_words))
    # arr = [list_top_words_and_count, count_words]

    # data_claster_common_words = pd.DataFrame((list_top_words_and_count))
    # columns=['Частые слова и их количество', 'Общее количество слов']
    
    # return data_claster_common_words # Не выгружается, слишком большой файл
    return list_top_words_and_count, count_words
    # return list_top_words_and_count

clasters_common_words = data_claster_out.groupby(column_id_group).apply(lambda x: top_terms(x, top_n=100))
count_clasters = len(clasters_common_words)
clasters_common_words

#  nums, words = map(list, zip(*nums_and_words)) 
words_info, count_words = map(list, zip(*clasters_common_words)) 
# data_clasters_common_words = pd.DataFrame([words_info, count_words], index=data_lemm_out[column_num], columns=['Частые слова и их количество', 'Общее количество слов'] )
frequent_words_count = 'Частые слова и их количество'
total_count_words = 'Общее количество слов'
data_clasters_common_words = pd.DataFrame(index=range(count_clasters), columns=[frequent_words_count, total_count_words] )
data_clasters_common_words[frequent_words_count] = words_info
data_clasters_common_words[total_count_words] = count_words
# data_clasters_common_words = pd.DataFrame([words_info, count_words])
data_clasters_common_words.tail(2)

,Частые слова и их количество,Общее количество слов
2065,"[(установка, 3), (бурильный, 3)]",6
2066,"[(установка, 4), (холодильный, 4)]",8


In [11]:
pd_clasters_common_words = pd.DataFrame(clasters_common_words)
pd_clasters_common_words.to_excel(f'{dir_data_out}/clasters_common_words.xlsx')

In [12]:
data_clasters_common_words[[frequent_words_count]]

,Частые слова и их количество
0,"[(nan, 2778), (изделие, 26), (предназначить, 1..."
1,"[(автоматический, 2), (установка, 2), (жидкост..."
2,"[(автоматический, 26), (устройство, 26), (упра..."
3,"[(автоматический, 3), (устройство, 3), (управл..."
4,"[(автопогрузчик, 6), (багажник, 1), (автомобил..."
...,...
2062,"[(средство, 5), (учебный, 5), (имитационный, 5..."
2063,"[(станок, 7), (бурение, 7), (скважина, 7), (го..."
2064,"[(взрывоопасный, 4), (оборудование, 3), (элект..."
2065,"[(установка, 3), (бурильный, 3)]"


In [13]:
data_clasters_common_words[:][[frequent_words_count]]

,Частые слова и их количество
0,"[(nan, 2778), (изделие, 26), (предназначить, 1..."
1,"[(автоматический, 2), (установка, 2), (жидкост..."
2,"[(автоматический, 26), (устройство, 26), (упра..."
3,"[(автоматический, 3), (устройство, 3), (управл..."
4,"[(автопогрузчик, 6), (багажник, 1), (автомобил..."
...,...
2062,"[(средство, 5), (учебный, 5), (имитационный, 5..."
2063,"[(станок, 7), (бурение, 7), (скважина, 7), (го..."
2064,"[(взрывоопасный, 4), (оборудование, 3), (элект..."
2065,"[(установка, 3), (бурильный, 3)]"


In [14]:
words, count_words = map(list,zip(*list(*data_clasters_common_words.loc[1:1, frequent_words_count])))


In [15]:
words, count_words = map(list,zip(*list(*data_clasters_common_words.loc[5:5, frequent_words_count])))
words

['фара',
 'огонь',
 'освещение',
 'задний',
 'мотоцикл',
 'галогенный',
 'лампа',
 'мопед',
 'свет',
 'фонарь',
 'габаритный',
 'противотуманный',
 'hs',
 'ближний',
 'дальний',
 'адаптивный',
 'система',
 'передний',
 'жгут',
 'проводы',
 'регистрационный',
 'знак',
 'указатель',
 'поворот',
 'контурный',
 'сигнал',
 'торможение',
 'устройство',
 'световой',
 'сигнализация',
 'квадрицикл',
 'ход',
 'транспортный',
 'средство',
 'hsb',
 'предупреждать',
 'стояночный',
 'дневный',
 'ходовой',
 'боковой',
 'газоразрядный',
 'источник',
 'автомобильный']

In [16]:
nltk.download('stopwords')
nltk.download('punkt')

stop_words = nltk.corpus.stopwords.words("russian")


def lemmatize(text_in):
    lemma = pymorphy2.MorphAnalyzer()
    text = " ".join([lemma.parse(word)[0].normal_form for word in text_in.split(' ')])
    tokens = word_tokenize(text, language="russian")
    for token in tokens:
        if token in stop_words:
            tokens.remove(token)
    text = " ".join(tokens)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rexarrior\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rexarrior\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
claster_words_precomputed = []
for id_claster in range(count_clasters):
    claster_words, claster_count_words = map(list,zip(*list(*data_clasters_common_words.loc[id_claster:id_claster, frequent_words_count]))) # Неудачные названия: claster_count_words, word_claster_count
    claster_words_precomputed.append({"claster_words":claster_words, "claster_count_words":claster_count_words})

import json
json_str = json.dumps(claster_words_precomputed)

with open(f'{dir_data_out}/data_to_str_vectors.json', "wt", encoding="utf8") as f:
    f.write(json_str)



In [18]:
print(total_count_words)
print(count_clasters)

Общее количество слов
2067


In [19]:
def get_metrics(product_name, claster_words_precomputed, clasters_common_words): # Брать 2 вариант
  product_name = lemmatize(product_name)
  product_name_words = product_name.split(' ')
  list_name_words = pd.unique(product_name_words)

  list_k = []
  for id_claster in range(count_clasters): # Внутри кластера суммировать или умножать - подумать    
    # print(range(count_clasters))
    claster_words = claster_words_precomputed[id_claster]['claster_words']
    claster_count_words = claster_words_precomputed[id_claster]['claster_count_words']
    claster_len = clasters_common_words.iloc[id_claster][total_count_words] # !!!
    # Tfidf - поискать наименование метрики - ??? 

    # print(f'Claster id: {str(id_claster)}')
    k = 0
    for word in list_name_words:  
      word_name_count = product_name_words.count(word)

      if word in claster_words:
        id_word = claster_words.index(word) 
        word_claster_count = claster_count_words[id_word] # !!!

        k += word_claster_count / (claster_len * len(list_name_words) / len(claster_words)) # !!!!! Подумать, как правильно !!!
        
    list_k.append(k)

  return list_k

row_ind = 800
product_name = all_data_in.iloc[row_ind]['Общее наименование продукции']
product_group = all_data_in.iloc[row_ind]['Группа продукции']
true_ind = all_data_in.iloc[row_ind]['Группа продукции_ordinal']
list_k = get_metrics(product_name, claster_words_precomputed, data_clasters_common_words)
max_coef = np.max(list_k)
ind  = list_k.index(max_coef)
choosen_product_group = all_data_in.iloc[row_ind]['Группа продукции']
print(f"max ind {ind} has {list_k[ind]}; true ind is {true_ind}; it's score {list_k[true_ind]}")
print(f"Product name: {product_name}")
print("True claster words:")
print(claster_words_precomputed[true_ind]['claster_words'])
print("Choosen claster words:")
print(claster_words_precomputed[ind]['claster_words'])



max ind 1733 has 0.9509534106546099; true ind is 826; it's score 0.45413570804195813
Product name: Котлы 4 категории, предназначенные для получения пара, избыточное давление которого до 3,0 МПа: Паровые котлы шотландского типа
True claster words:
['иметь', 'вместимость', 'м', 'свыше', 'котёл', 'предназначить', 'получение', 'горячий', 'вода', 'температура', 'или', 'пара', 'избыточный', 'давление', 'мпа', 'также', 'сосуд', 'огнев', 'обогрев', 'категория', 'светильник', 'общий', 'назначение', 'игрушка', 'прочий']
Choosen claster words:
['рабочий', 'предназначить', 'среда', 'газ', 'давление', 'иметь', 'максимально', 'допустимый', 'свыше', 'мпа', 'использовать', 'категория', 'пар', 'группа', 'трубопровод', 'номинальный', 'диаметр', 'мм', 'взрывоопасный', 'изделие', 'оборудование', 'средство', 'перевозка', 'электрооборудование', 'эксплуатация', 'морской', 'они', 'ядерный', 'оружие', 'электрический', 'назначение', 'транспортный', 'общий', 'пассажир', 'неэлектрический', 'работа', 'применять', 

In [20]:
data_k = pd.DataFrame(list_k, index=range(count_clasters), columns=['Коэф. k'])
data_k

,Коэф. k
0,0.034200
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
2062,0.000000
2063,0.000000
2064,0.106183
2065,0.000000


In [21]:
short_dataset = all_data_in.head(100)
additional_columns = ["cl." + str(i) for i in range(count_clasters)]
extended_short_dataset = pd.DataFrame(columns=additional_columns)
for i in range(short_dataset.shape[0]):
    print(f"process {i} row")
    row = short_dataset.iloc[i]
    list_k = get_metrics(row['Общее наименование продукции'], claster_words_precomputed, data_clasters_common_words)
    data_k = pd.DataFrame([list_k], index=range(1), columns=additional_columns)
    extended_short_dataset = extended_short_dataset.append(data_k)
    pass
extended_short_dataset.tail(5)

process 0 row
process 1 row
process 2 row
process 3 row
process 4 row
process 5 row
process 6 row
process 7 row
process 8 row
process 9 row
process 10 row
process 11 row
process 12 row
process 13 row
process 14 row
process 15 row
process 16 row
process 17 row
process 18 row
process 19 row
process 20 row
process 21 row
process 22 row
process 23 row
process 24 row
process 25 row
process 26 row
process 27 row
process 28 row
process 29 row
process 30 row
process 31 row
process 32 row
process 33 row
process 34 row
process 35 row
process 36 row
process 37 row
process 38 row
process 39 row
process 40 row
process 41 row
process 42 row
process 43 row
process 44 row
process 45 row
process 46 row
process 47 row
process 48 row
process 49 row
process 50 row
process 51 row
process 52 row
process 53 row
process 54 row
process 55 row
process 56 row
process 57 row
process 58 row
process 59 row
process 60 row
process 61 row
process 62 row
process 63 row
process 64 row
process 65 row
process 66 row
proce

,cl.0,cl.1,cl.2,cl.3,cl.4,cl.5,cl.6,cl.7,cl.8,cl.9,...,cl.2057,cl.2058,cl.2059,cl.2060,cl.2061,cl.2062,cl.2063,cl.2064,cl.2065,cl.2066
0,0.05472,0,0,0.0,0,0.0,0.0,0,0,0,...,0.0,0.0,0,0,0.0,0,0,0.0,0,0
0,0.05472,0,0,0.0,0,0.0,0.0,0,0,0,...,0.0,0.0,0,0,0.0,0,0,0.0,0,0
0,0.05472,0,0,0.0,0,0.0,0.0,0,0,0,...,0.0,0.0,0,0,0.0,0,0,0.0,0,0
0,0.05472,0,0,0.0,0,0.0,0.0,0,0,0,...,0.0,0.0,0,0,0.0,0,0,0.0,0,0
0,0.05472,0,0,0.0,0,0.0,0.0,0,0,0,...,0.0,0.0,0,0,0.0,0,0,0.0,0,0


In [24]:
dataset = pd.merge(short_dataset, extended_short_dataset, on=[short_dataset.index])
dataset.head(5)

ValueError: Unable to fill values because RangeIndex cannot contain NA

In [23]:

dataset.to_excel(f'{dir_data_out}/short_extended_dataset.xlsx')

KeyboardInterrupt: 

In [142]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2)
features = [additional_columns]
target = ['is_valid']

In [143]:
X_train = train[features]
Y_train = train[target]
X_test = test[features]
Y_test = test[target]

KeyError: "None of [Index([('cl.0', 'cl.1', 'cl.2', 'cl.3', 'cl.4', 'cl.5', 'cl.6', 'cl.7', 'cl.8', 'cl.9', 'cl.10', 'cl.11', 'cl.12', 'cl.13', 'cl.14', 'cl.15', 'cl.16', 'cl.17', 'cl.18', 'cl.19', 'cl.20', 'cl.21', 'cl.22', 'cl.23', 'cl.24', 'cl.25', 'cl.26', 'cl.27', 'cl.28', 'cl.29', 'cl.30', 'cl.31', 'cl.32', 'cl.33', 'cl.34', 'cl.35', 'cl.36', 'cl.37', 'cl.38', 'cl.39', 'cl.40', 'cl.41', 'cl.42', 'cl.43', 'cl.44', 'cl.45', 'cl.46', 'cl.47', 'cl.48', 'cl.49', 'cl.50', 'cl.51', 'cl.52', 'cl.53', 'cl.54', 'cl.55', 'cl.56', 'cl.57', 'cl.58', 'cl.59', 'cl.60', 'cl.61', 'cl.62', 'cl.63', 'cl.64', 'cl.65', 'cl.66', 'cl.67', 'cl.68', 'cl.69', 'cl.70', 'cl.71', 'cl.72', 'cl.73', 'cl.74', 'cl.75', 'cl.76', 'cl.77', 'cl.78', 'cl.79', 'cl.80', 'cl.81', 'cl.82', 'cl.83', 'cl.84', 'cl.85', 'cl.86', 'cl.87', 'cl.88', 'cl.89', 'cl.90', 'cl.91', 'cl.92', 'cl.93', 'cl.94', 'cl.95', 'cl.96', 'cl.97', 'cl.98', 'cl.99', ...)], dtype='object')] are in the [columns]"

In [ ]:

from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import math
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

def plot_metrics_for_logreg(model, X_test, y_test):
    logit_roc_auc = roc_auc_score(y_test, model.predict(X_test))
    fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")


def logisticregression_classifier(base_x, base_y, is_need_plot=True, is_scaled=True, test_percent=0.2):
    model = LogisticRegression(penalty="l2", solver="sag") # full 0.7333
    # logreg = LogisticRegression(penalty="elasticnet", solver="saga", l1_ratio=0.5) # full 0.7333
    if (is_scaled):
        base_x = normalize(base_x, "logistic")
    X_train, X_test, y_train, y_test = train_test_split(base_x, base_y, test_size=test_percent, random_state=0)
   
    model.fit(X_train, y_train)
    result_score = model.score(X_test, y_test)
    print('Accuracy of logistic regression classifier on train set: {:.4f}'.format(model.score(X_train, y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.4f}'.format(result_score))
    if (is_need_plot):
        plt.plot_metrics_for_logreg(model, X_test, y_test)
        plt.show()
    return model, result_score


def randomforest_classifier(base_x, base_y, is_need_plot=True,  is_scaled=False, test_percent=0.3):
    model = RandomForestClassifier(n_estimators=1000, criterion="gini")
    if (is_scaled):
        base_x = normalize(base_x, "rforest")
    X_train, X_test, y_train, y_test = train_test_split(base_x, base_y, test_size=test_percent, random_state=0)
   
    model.fit(X_train, y_train)
    result_score = model.score(X_test, y_test)
    print('Accuracy of regression on train set: {:.4f}'.format(model.score(X_train, y_train)))
    print('Accuracy of regression on test set: {:.4f}'.format(model.score(X_test, y_test)))
    return model, result_score, model.feature_importances_




def randomforest_regression(base_x, base_y, is_scaled=False, test_percent=0.3):
    model = RandomForestRegressor(n_estimators=1000, verbose=1)
    if (is_scaled):
        base_x = normalize(base_x, "rforest")
        base_y = normalize(base_y, "rforest")
    X_train, X_test, y_train, y_test = train_test_split(base_x, base_y, test_size=test_percent, random_state=0)
   
    model.fit(X_train, y_train)
    result_score = model.score(X_test, y_test)
    print('Accuracy of regression on train set: {:.4f}'.format(model.score(X_train, y_train)))
    print('Accuracy of regression on test set: {:.4f}'.format(model.score(X_test, y_test)))
    return model, result_score, model.feature_importances_



def normalize(data, type="logistic"):
    if (type == "logistic" or "rforest"):
        scaler = preprocessing.Normalizer().fit(data)
    elif(type == "linear"):
        scaler = preprocessing.StandardScaler().fit(data)
    scaled =  scaler.transform(data)
    return scaled





In [ ]:
logistic_model, train_score = logisticregression_classifier(X_train, Y_train, True)

In [ ]:
result = logistic_model.predict(X_test)
from sklearn.metrics import f1_score
f1_score(Y_test, result)